In [2]:
import matplotlib, os, sys
import matplotlib.font_manager
from matplotlib import rcParams

#import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from scipy.io import loadmat
from pathlib import Path
from scipy.stats import mode
from scipy.interpolate import interp1d, RectBivariateSpline, interp2d, CloughTocher2DInterpolator, interpn
from scipy.integrate import cumtrapz
from scipy.optimize import minimize

sys.path.append('../../..')
sys.path.append('../..')
sys.path.append('../')

from tqdm import tqdm
from setup import *
from modules.Espec import *#get_raw_image
from modules.GammaSpec.GammaSpecProc import *
from lib.general_tools import imagesc, load_object, calc_COW, d, smooth_gauss, compute_percentile_from_dist
from lib.general_tools import choose_cal_file, glob_path, get_file_path, save_object
from lib.analysis_database import Database

plt.style.use('/home/el1103292/GitHub/RadiationReaction2021Analysis/lib/thesis.mplstyle')
DATA_PATH =Path(ROOT_DATA_FOLDER)
DATA_PATH

PosixPath('/data/instrument/GEMINI/2021/App20110008-1/MIRAGE')

In [3]:
# reads gamma profile analysis
analysis_path = Path(r'/data/analysis/GEMINI/2021/App20110008-1/Results/')
diag = 'GammaProfile'
database_file = analysis_path / f'{diag}_results_main.pkl'
gammaProf_db = Database(database_file,always_load=True)
gammaProf_db.dataframe.head()

img_med  img_max
run            shot                  
20210604/run02 1        99.0    111.0
               2        99.0    108.0
               3        98.0    111.0
               4        99.0    119.0
               5        99.0    120.0

In [5]:
def compute_percentile_from_dist(dist, axis, target_percentile):
    
    div=10.0
    N=int(len(dist)/div)
    percentile, energy=np.zeros(N), np.zeros(N)
    dist=dist/np.trapz(dist, axis)
    for i in range(0, N):
        percentile[i]=np.trapz(dist[0:len(axis)-1-int(div)*i], axis[0:len(axis)-1-int(div)*i])
        energy[i]=axis[len(axis)-1-int(div)*i]
        if percentile[i]<target_percentile-0.005:
            break
    percentile_cut=percentile[percentile!=0.0]
    energy_cut=energy[energy!=0.0]
    energy_at_percentile=np.interp(target_percentile, percentile_cut, energy_cut)
    return energy_at_percentile

def compute_peakiness_measure(dQ_dE, E_MeV, color, normalisation_factor_E):
    E_GeV=E_MeV/1000.0
    dQ_dE=dQ_dE*E_GeV/np.trapz(dQ_dE*E_GeV, E_GeV)
    Q_tot=np.trapz(dQ_dE, E_GeV)
    mean_energy_tmp=np.trapz(dQ_dE*E_GeV, E_GeV)/Q_tot
    mean_energy_sqrd_tmp=np.trapz(dQ_dE*E_MeV*E_GeV, E_GeV)/Q_tot
    
    e_at_70th_percentile=compute_percentile_from_dist(dQ_dE, E_GeV, 0.7)
    e_at_50th_percentile=compute_percentile_from_dist(dQ_dE, E_GeV, 0.5)
    E_thresh=e_at_70th_percentile
    indx_near_mean=np.argmin(abs(E_GeV-E_thresh))

    mean_dQ_dE=np.mean(dQ_dE)
    max_over_med_above_thresh=max(dQ_dE[indx_near_mean:])#(max(dQ_dE[indx_near_mean:])-mean_dQ_dE)/mean_dQ_dE
    peakiness_measure=max_over_med_above_thresh
    return peakiness_measure, E_thresh, max(dQ_dE[indx_near_mean:])#peakiness_measure, mean_energy_tmp
    
def get_electron_props(gammaProf_db, get_e_img=False):
    e_img_list = []
    E_90th_percentile=[]
    Qgamma2=[]
    av_gamma2=[]
    peakiness_arr=[]
    Q_energy_thresh=[]
    electron_mean_E=[]
    Q_tot=[]
    time_arr=[]
    
    count=0
    color_arr=['r', 'b', 'g', 'orange', 'cyan', 'magenta']
    
    time_path='/home/el1103292/GitHub/RadiationReaction2021Analysis/lib/shot_2_GSN/'
    times_file=time_path+'%s.pkl'%(run_name[0:8])
    run_no='run18'
    times_data = load_object(times_file)
    times_data_current_run=times_data[run_no]
    all_shot=np.array(times_data_current_run['Shot numbers'])
    all_times=np.array(times_data_current_run['Time'])
    print(all_shot)
    print(all_times)
    times_data_current_run01=times_data['run01']
    all_shotr1=np.array(times_data_current_run['Shot numbers'])
    gsnr1=np.array(times_data_current_run['GSN'])
    all_timesr1=np.array(times_data_current_run['Time'])
    print(gsnr1[0])
    
    
    
#     for n,row in gammaProf_db.dataframe.iterrows():
#         run_name,shot = row.name
#         spectra_path = Path('/data/analysis/GEMINI/2021/App20110008-1/Results/Espec/Spectra') / run_name.replace('/','_')
#         espec_file = spectra_path / f'Shot{shot:04}_spectra.pkl'
        
        
        
#         #get time of shot
        
#         normalisation_factor_E=np.random.uniform(800.0, 1500.0)
#         time_path='/home/el1103292/GitHub/RadiationReaction2021Analysis/lib/shot_2_GSN/'
#         times_file=time_path+'%s.pkl'%(run_name[0:8])
#         run_no=run_name[9:]
#         times_data = load_object(times_file)
#         print('run_no=%s'%run_no)
#         times_data_current_run=times_data[run_no]
#         all_shot=times_data_current_run['Shot numbers']
#         all_times=times_data_current_run['Time']
#         print('all_shot.shape=%s'%[all_shot.shape])
#         print('all_times.shape=%s'%[all_times.shape])
#         print('shot=%s'%shot)
#         idx_shot=np.where(all_shot==shot)
#         print('idx_shot=%s'%idx_shot)
#         current_time=all_times[idx_shot]
#         print('current_time=%s'%current_time)
#         time_arr.append(current_time)
#         current_shot=all_shot[idx_shot]
# #         print(current_shot)
# #         print(current_time)
        
#         if os.path.isfile(espec_file):
#             espec_data = load_object(espec_file)
#             E_MeV,y_mrad,spec1, dWdE1_JperGeV = espec_data['E_MeV'],espec_data['y1_mrad'],espec_data['spec1'],espec_data['dWdE1_JperGeV']
#             if get_e_img==True:
#                 y0 = y_mrad[np.argmax(np.sum(spec1,axis=1))]
#                 y_mrad_disp = np.linspace(-4.5,4.5,100)
#                 spec_img = RectBivariateSpline(y_mrad-y0,E_MeV,spec1)(y_mrad_disp,E_MeV)
#                 e_img_list.append(spec_img)
                
#             #90th percentile e
#             e_at_90th_percentile=compute_percentile_from_dist(dWdE1_JperGeV, E_MeV/1000.0, 0.9)
#             E_90th_percentile.append(e_at_90th_percentile)

#             #charge 
#             dQ_dE=np.trapz(spec1,x=y_mrad,axis=0)
#             Q_tot_tmp=np.trapz(dQ_dE, E_MeV)
#             Q_tot.append(Q_tot_tmp)

#             #Q gamma_sqrd
#             Qgamma2_tmp=np.trapz(dQ_dE*E_MeV/0.511*E_MeV/0.511, E_MeV)
#             Qgamma2.append(Qgamma2_tmp)
#             av_gamma2.append(Qgamma2_tmp/Q_tot_tmp)

#             #mean energy per electron
#             electron_mean_E_tmp=np.trapz(dQ_dE*(E_MeV), E_MeV)/Q_tot_tmp
#             electron_mean_E.append(electron_mean_E_tmp)

#             #peakiness metric
#             peakiness, energy_thresh_tmp, max_height=compute_peakiness_measure(dQ_dE, E_MeV, "r", normalisation_factor_E)
#             Q_energy_thresh.append(energy_thresh_tmp)
#             peakiness_arr.append(peakiness)

#         #             if count<6:
#         #                 plt.plot(np.full(100, energy_thresh_tmp/rand_norm), np.linspace(0.0, 15.0, 100), color=color_arr[count])
#         #                 E_GeV=E_MeV/(1000.0*rand_norm)
#         #                 dEW_DE=dQ_dE*E_GeV/np.trapz(dQ_dE*E_GeV, E_GeV)
#         #                 plt.plot(E_GeV, dEW_DE, color=color_arr[count])
#         #                 count+=1
# #         else:
# #             e_img_list.append(spec_img*np.nan)
# #             E_90th_percentile_hits.append(np.nan)
# #     plt.xticks(fontsize=12)
# #     plt.yticks(fontsize=12)
# #     plt.xlabel("Blinded Energy (a.u.)", fontsize=16)
# #     plt.ylabel("Normalised Charge per\n Relative Energy Spread (a.u.)", fontsize=16)
#     print(np.array(E_90th_percentile).shape)
#     return np.array(e_img_list), np.array(E_90th_percentile), np.array(Qgamma2), np.array(av_gamma2), np.array(peakiness_arr), np.array(Q_energy_thresh), np.array(electron_mean_E), np.array(Q_tot), np.array(time_arr)


# np.random.seed(67)
# rand_norm=np.random.uniform(0.1, 10.0)



# e_img_list, E_90th_percentile, Qgamma2_hits, av_gamma2_hits, peakiness_arr, Q_energy_thresh, electron_mean_E, Q_tot, times=get_electron_props(gammaProf_db, False)




In [20]:
plt.plot(times, Q_tot)
plt.show()

NameError: name 'times' is not defined